In [ ]:
!pip install -U  diffusers[torch] transformers accelerate bitsandbytes
!pip install -U "huggingface_hub[cli]"

In [ ]:
!huggingface-cli login --token HF_TOKEN

In [ ]:
import torch
from diffusers import FluxPipeline

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

pipe = FluxPipeline.from_pretrained(
    "043dumb/flux_fp4_model",
    torch_dtype=torch.float16,
    cache_dir ="./flux_model"
).to(device)

In [ ]:
!pip install "fastapi[standard]" nest-asyncio pyngrok uvicorn
!ngrok config add-authtoken "NGROK_AUTHENTICATION_CODE"

In [ ]:
from pydantic import BaseModel
from fastapi import FastAPI
import base64
import os
import io

def encode(input_image):
    # Save to a temporary in-memory file instead of disk
    buffered = io.BytesIO()
    input_image.save(buffered, format="PNG")  # or "JPEG", depending on the image
    encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    return encoded_string

app = FastAPI()

def inference(request):
    try:
        # Define pipeline parameters
        pipe_kwargs = {
            "prompt": request.prompt,
            "negative_prompt": request.neg_prompt,
            "num_inference_steps": request.steps,
            "height": request.height,
            "width": request.width,
        }

        # Generate image
        image = pipe(**pipe_kwargs).images[0]

        encoded_string = encode(image)

        # Clear VRAM
        del pipe_kwargs
        torch.cuda.empty_cache()

        return encoded_string

    except Exception as e:
        return f"Error: {str(e)}"

# BaseModel lets use define the request body format
class ImageRequest(BaseModel):
    prompt: str
    neg_prompt: str
    steps: int
    height: int
    width: int

@app.post("/")
def generate_image(request: ImageRequest):
    encoded_string = inference(request)

    # returning image as base64
    return {"image": encoded_string}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)